# Amazon Bedrock boto3 Setup

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

In this demo notebook, we demonstrate how to use the [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with [Amazon Bedrock](https://aws.amazon.com/bedrock/) Foundation Models.

---

## Prerequisites

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


  Using cached boto3-1.34.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached awscli-1.32.7-py3-none-any.whl.metadata (11 kB)
  Using cached botocore-1.34.7-py3-none-any.whl.metadata (5.6 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
Using cached boto3-1.34.7-py3-none-any.whl (139 kB)
Using cached awscli-1.32.7-py3-none-any.whl (4.3 MB)
Using cached botocore-1.34.7-py3-none-any.whl (11.9 MB

In [ ]:
%pip install --quiet \
    langchain==0.0.309 \
    matplotlib


In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

---

## Create the boto3 client

Interaction with the Bedrock API is done via the AWS SDK for Python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).

Depending on your environment, you might need to customize the setup when creating your Bedrock service client. To help with this, we've provided a `get_bedrock_client()` utility method that supports passing in different options. You can find the implementation in [../utils/bedrock.py](../utils/bedrock.py)

#### Use different clients
The boto3 provides different clients for Amazon Bedrock to perform different actions. The actions for [`InvokeModel`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) and [`InvokeModelWithResponseStream`](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModelWithResponseStream.html) are supported by Amazon Bedrock Runtime where as other operations, such as [ListFoundationModels](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_ListFoundationModels.html), are handled via [Amazon Bedrock client](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock.html).

The `get_bedrock_client()` method accepts `runtime` (default=True) parameter to return either `bedrock` or `bedrock-runtime` client.

#### Use the default credential chain

If you are running this notebook from [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) and your Sagemaker Studio [execution role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) has permissions to access Bedrock you can just run the cells below as-is. This is also the case if you are running these notebooks from a computer whose default AWS credentials have access to Bedrock.

#### Use a different AWS Region

If you're running this notebook from your own computer or a SageMaker notebook in a different AWS Region from where Bedrock is set up, you can un-comment the `os.environ['AWS_DEFAULT_REGION']` line below and specify the region to use.

#### Use a specific profile

In case you're running this notebook from your own computer where you have setup the AWS CLI with multiple profiles, and the profile which has access to Bedrock is not the default one, you can un-comment the `os.environ['AWS_PROFILE']` line below and specify the profile to use.

#### Use a different role

In case you or your company has setup a specific, separate [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) to access Bedrock, you can specify it by un-commenting the `os.environ['BEDROCK_ASSUME_ROLE']` line below. Ensure that your current user or role have permissions to [assume](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html) such role.

#### A note about `langchain`

The Bedrock classes provided by `langchain` create a Bedrock boto3 client by default. To customize your Bedrock configuration, we recommend to explicitly create the Bedrock client using the method below, and pass it to the [`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) class instantiation method using `client=boto3_bedrock`

In [4]:
import json
import os
import sys

import boto3
import botocore

import bedrock


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)



Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


---

## Common inference parameter definitions

### Randomness and Diversity

Foundation models support the following parameters to control randomness and diversity in the 
response.

**Temperature** – Large language models use probability to construct the words in a sequence. For any 
given next word, there is a probability distribution of options for the next word in the sequence. When 
you set the temperature closer to zero, the model tends to select the higher-probability words. When 
you set the temperature further away from zero, the model may select a lower-probability word.

In technical terms, the temperature modulates the probability density function for the next tokens, 
implementing the temperature sampling technique. This parameter can deepen or flatten the density 
function curve. A lower value results in a steeper curve with more deterministic responses, and a higher 
value results in a flatter curve with more random responses.

**Top K** – Temperature defines the probability distribution of potential words, and Top K defines the cut 
off where the model no longer selects the words. For example, if K=50, the model selects from 50 of the 
most probable words that could be next in a given sequence. This reduces the probability that an unusual 
word gets selected next in a sequence.
In technical terms, Top K is the number of the highest-probability vocabulary tokens to keep for Top-
K-filtering - This limits the distribution of probable tokens, so the model chooses one of the highest-
probability tokens.

**Top P** – Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top 
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is 
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their 
probabilities.
For the example prompt "I hear the hoof beats of ," you may want the model to provide "horses," 
"zebras" or "unicorns" as the next word. If you set the temperature to its maximum, without capping 
Top K or Top P, you increase the probability of getting unusual results such as "unicorns." If you set the 
temperature to 0, you increase the probability of "horses." If you set a high temperature and set Top K or 
Top P to the maximum, you increase the probability of "horses" or "zebras," and decrease the probability 
of "unicorns."

### Length

The following parameters control the length of the generated response.

**Response length** – Configures the minimum and maximum number of tokens to use in the generated 
response.

**Length penalty** – Length penalty optimizes the model to be more concise in its output by penalizing 
longer responses. Length penalty differs from response length as the response length is a hard cut off for 
the minimum or maximum response length.

In technical terms, the length penalty penalizes the model exponentially for lengthy responses. 0.0 
means no penalty. Set a value less than 0.0 for the model to generate longer sequences, or set a value 
greater than 0.0 for the model to produce shorter sequences.

### Repetitions

The following parameters help control repetition in the generated response.

**Repetition penalty (presence penalty)** – Prevents repetitions of the same words (tokens) in responses. 
1.0 means no penalty. Greater than 1.0 decreases repetition.

In [148]:
# helper functions

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

# return the text from an html page
def get_html_text(url, postprocess=False, print_text=False):

    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    if postprocess is True:
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

    if print_text is True:
        print(text)
    
    return text

# parsing completion
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def display_graph(graph, show_link=False):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    url_rendering = "https://mermaid.ink/img/" + base64_string
    if show_link is True:
        print(url_rendering)
    rendered_graph = Image(url=url_rendering)
    display(rendered_graph)

    
def process_completion(llm_completion):
    str_mermaid_graph = find_between(llm_completion, "<mermaid>", "</mermaid>")
    print(str_mermaid_graph)
    if check_graph_validity(str_mermaid_graph) is True:
        display_graph(str_mermaid_graph)
    else:
        print("!!!The diagram has syntax issues!!!")
    
    
def check_graph_validity(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    url = "https://mermaid.ink/img/" + base64_string
    req = Request(url, headers={'User-Agent' : "Magic Browser"})
    
    flag_valid = True
    try: 
        con = urlopen(req)
    except:
        flag_valid = False
    
    return flag_valid
        

    
def standardize_graph(graph):
    
    # double to single
    graph = graph.replace("((", "(")
    graph = graph.replace("))", ")")
    graph = graph.replace("{{", "{")
    graph = graph.replace("}}", "}")
    graph = graph.replace("[[", "[")
    graph = graph.replace("]]", "]")
    graph = graph.replace("||", "|")
    
    graph = graph.replace("([", "(")
    graph = graph.replace("])", ")")
    graph = graph.replace("[(", "[")
    graph = graph.replace(")]", "]")
    graph = graph.replace("({", "(")
    graph = graph.replace("})", ")")
    graph = graph.replace("{(", "{")
    graph = graph.replace(")}", "}")
    graph = graph.replace("{[", "{")
    graph = graph.replace("]}", "}")
    graph = graph.replace("[{", "[")
    graph = graph.replace("}]", "]")
    
    # remove quotes if they exist (to be added later)
    graph = graph.replace('("', '(')
    graph = graph.replace('")', ')')
    graph = graph.replace('["', '[')
    graph = graph.replace('"]', ']')
    graph = graph.replace('{"', '{')
    graph = graph.replace('"}', '}')
    
    # add quotes
    graph = graph.replace('(', '("')
    graph = graph.replace(')', '")')
    graph = graph.replace('[', '["')
    graph = graph.replace(']', '"]')
    graph = graph.replace('{', '{"')
    graph = graph.replace('}', '"}')
    
    # # remove spaces in arrows (to be added later)
    # graph = graph.replace(' -->', '-->')
    # graph = graph.replace('--> ', '-->')
    # graph = graph.replace('-->', ' --> ')
    
    # remove unsafe caracters for base64 encoding
    graph = graph.replace('/', '')
    graph = graph.replace('+', '')
    
    return graph


In [149]:
def generate_diagram(
    url,
    number_of_diagrams=1,
    kind="flowchart", # "mindmap" or "flowchart"
    orientation="LR", # "LR" or "TD"
    mermaid_context=True,
    max_tokens_to_sample=500,
    temperature=0.9,
    top_k=250,
    top_p=1,
    show_raw_result=True,
    display=True
):
    ls_diagrams=[]
    
    for d in range(number_of_diagrams):
        print("--- Generating diagram", d+1, "out of", number_of_diagrams, "---")
        
        print("...Getting context...")
        html_text = context_mermaid_notation = get_html_text(
            url=url, 
            postprocess=False, 
            print_text=False
        )

        if mermaid_context is True:
            context_mermaid_notation = get_html_text(
                url="https://mermaid.js.org/syntax/flowchart.html", 
                postprocess=False, 
                print_text=False
            )
        else:
            context_mermaid_notation = ""


        prompt = f"""\n\nHuman: 
        Here is a text for you to reference for the following task:
        <text>
        {html_text}
        </text>

        Task: Summarize the given text and provide the summary inside <summary> tags. 
        Then convert the summary to a {kind} using Mermaid notation. 

        <mermaid_notation>
        {context_mermaid_notation}
        </mermaid_notation>

        The {kind} should capture the main gist of the summary, without too many low-level details. 
        Someone who would only view the Mermaid {kind}, should understand the gist of the summary. 
        The Mermaid {kind} should follow all the correct notation rules and should compile without any errors.
        Use the following specifications for the generated Mermaid {kind}:

        <specifications>
        1. Use different colors, shapes or groups to represent different concepts in the given text.
        2. The orientation of the Mermaid {kind} should be {orientation}.
        3. Any text inside parenthesis (), square brackets [], curly brackets {{}}, or bars ||, should be inside quotes "".
        4. Include the Mermaid {kind} inside <mermaid> tags.
        5. Do not write anything after the </mermaid> tag.
        6. Use only information from within the given text. Don't make up new information.
        </specifications>

        \n\nAssistant:
        """

        prompt = prompt.replace("{{}}", "{}")
        
        print("...LLM is generating...") 
        body = json.dumps(
            {
                "prompt": prompt, 
                "max_tokens_to_sample": max_tokens_to_sample,
                "temperature": temperature,
                "top_k": top_k,
                "top_p": top_p,
                "stop_sequences": ["\n\nHuman:"]
            }
        )
        modelId = "anthropic.claude-v2:1"  # change this to use a different version from the model provider
        accept = "application/json"
        contentType = "application/json"

        response = bedrock_runtime.invoke_model(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
        response_body = json.loads(response.get("body").read())

        print("...Displaying graph...")

        if show_raw_result is True:
            print(response_body.get("completion"))

        str_mermaid_graph = find_between(
            response_body.get("completion"), 
            "<mermaid>", 
            "</mermaid>"
        )
        
        dc_output = {}
        dc_output["raw"] = response_body.get("completion")
        dc_output["graph"] = str_mermaid_graph
        dc_output["standardized_graph"] = standardize_graph(str_mermaid_graph)
        # dc_output["graph"] = str_mermaid_graph.replace(",", "")  # beause https://mermaid.ink/ raises an error! (why?)
        dc_output["valid"] = check_graph_validity(str_mermaid_graph)
        ls_diagrams.append(dc_output)
        
        if display is True:
            print(dc_output["standardized_graph"])
            if check_graph_validity(dc_output["standardized_graph"]) is True:
                display_graph(dc_output["standardized_graph"])
            else:
                print("!!!The diagram has syntax issues!!!\n")
    
    return ls_diagrams

In [150]:
ls_diagrams = generate_diagram(
    # url="https://docs.aws.amazon.com/sagemaker/latest/dg/onboard-vpc.html",
    url="https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers.html",
    # url="https://code.likeagirl.io/creating-flowcharts-with-mermaid-in-python-3cbca0058ecb",
    number_of_diagrams=3,
    kind="flowchart", # "mindmap" or "flowchart"
    orientation="LR", # "LR" or "TD"
    mermaid_context=True,
    max_tokens_to_sample=500,
    temperature=0.9,
    top_k=250,
    top_p=1,
    show_raw_result=False,
    display=True
)

--- Generating diagram 1 out of 3 ---
...Getting context...
...LLM is generating...
...Displaying graph...

flowchart LR
    direction LR
    C1["Custom model with <br>custom framework"] --> A1("Build <br>own container")
    C2["Custom model <br>needing custom <br>package"] --> B2["Extend built-in <br>container"]
    C3["Custom model <br>supported <br>framework"] --> B1{"Use built-in <br>container as is"}
    C4["Pre-built <br>algorithm model"] --> B3["Use pre-built <br>container"]



--- Generating diagram 2 out of 3 ---
...Getting context...
...LLM is generating...
...Displaying graph...

flowchart LR
    direction LR
    A["Use Cases"] --> B{"Need<br>custom<br>packages?"}
    B --> |No| C["Use pre-built<br>container"]
    B --> |Yes| D{"Pre-built<br>container<br>supports<br>requirements.txt?"} 
    D --> |Yes| E["Add dependencies<br>in requirements.txt"]
    D --> |No| F["Extend pre-built<br>container"]
    F --> C
    C --> G["Train &<br>deploy<br>model"]

    classDef grey fill:#ddd,stroke:#fff,stroke-width:4px,color:#000
    class A,G grey

!!!The diagram has syntax issues!!!

--- Generating diagram 3 out of 3 ---
...Getting context...
...LLM is generating...
...Displaying graph...

!!!The diagram has syntax issues!!!



In [158]:
mermaid_code = ls_diagrams[1]["standardized_graph"]
mermaid_code=mermaid_code.replace(
    'A["Use Cases"] --> B{"Need<br>custom<br>packages?"}', 
    'A["Use Cases"] --> B{"Need<br>custom<br>packages ?"}'
)

# mermaid_code=mermaid_code[:mermaid_code.find('B --> |No|')]
print(repr(mermaid_code))
print((mermaid_code))

display_graph(mermaid_code, show_link=True)

'\nflowchart LR\n    direction LR\n    A["Use Cases"] --> B{"Need<br>custom<br>packages ?"}\n    B --> |No| C["Use pre-built<br>container"]\n    B --> |Yes| D{"Pre-built<br>container<br>supports<br>requirements.txt?"} \n    D --> |Yes| E["Add dependencies<br>in requirements.txt"]\n    D --> |No| F["Extend pre-built<br>container"]\n    F --> C\n    C --> G["Train &<br>deploy<br>model"]\n\n    classDef grey fill:#ddd,stroke:#fff,stroke-width:4px,color:#000\n    class A,G grey\n'

flowchart LR
    direction LR
    A["Use Cases"] --> B{"Need<br>custom<br>packages ?"}
    B --> |No| C["Use pre-built<br>container"]
    B --> |Yes| D{"Pre-built<br>container<br>supports<br>requirements.txt?"} 
    D --> |Yes| E["Add dependencies<br>in requirements.txt"]
    D --> |No| F["Extend pre-built<br>container"]
    F --> C
    C --> G["Train &<br>deploy<br>model"]

    classDef grey fill:#ddd,stroke:#fff,stroke-width:4px,color:#000
    class A,G grey

https://mermaid.ink/img/CmZsb3djaGFydCBMUgogICAgZGl